In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
from matplotlib import lines as mlines
rcParams["savefig.dpi"] = 300
rcParams["figure.dpi"] = 300

In [ ]:
import matplotlib as mpl
default_colors = mpl.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json, os, pickle, glob
import tqdm
import corner
import scipy.linalg as sl
import logging
logger = logging.getLogger(__name__)

In [ ]:
from scipy.linalg import LinAlgError
import la_forge.diagnostics as dg
import la_forge.core as co
from la_forge.rednoise import gorilla_bf
from la_forge.utils import epoch_ave_resid
from h5pulsar import FilePulsar
from enterprise.signals.utils import ConditionalGP
#from la_forge.gp import Signal_Reconstruction
from enterprise.signals import parameter, signal_base, utils, gp_signals, selections
from enterprise_extensions import chromatic as chrom
import enterprise.constants as const
from enterprise_extensions.empirical_distr import EmpiricalDistribution1D, EmpiricalDistribution2D
from collections import OrderedDict
DM_K = float(2.41e-4)

In [ ]:
from IPTA_DR2_analysis.model_blocks import adv_noise_block

In [ ]:
from dr3_noise import post_processing_utils as ppu
from dr3_noise import plot_utils as pu
from dr3_noise.models import model_singlepsr_noise

## Compare the noise properties of lite vs lite-combined vs full

Some things to make include the following:
- Done: Overplot white noise and red noise parameters for each pulsar (1D hists)
- Done: Overplot red noise parameters for each pulsar (corners)
- Done: Overplot GP realizations for each pulsar
    - Overplot GPs with adjustments (upper limit draws, same Tspans)
- Compare all pulsars with significant red noise (all psrs 68% intervals on the same plot)
    - lite pulsars: labelled, colored
    - lite-combined pulsars: labelled, colored, lighter alpha
    - full pulsars: grey, unlabelled, lighter alpha
- Same comparison plot as above for DM
- Intrisic noise comparison using full PTA CRN search
    - Could also try this with CRN 13/3, CRN varied gamma, HD 13/3, or HD varied gamma (latter is probably best)

In [ ]:
def make_labels(params, psrname, add_psrname=True):
    labels = []
    if add_psrname:
        add = psrname+'\n'
    else:
        add = ''
    for p in params:
        if 'dm_gp_gamma' in p:
            labels.append(add+r'$\gamma_{\rm{DM}}$')
        elif 'dm_gp_log10_A' in p:
            labels.append(add+r'$\log_{10}A_{\rm{DM}}$')
        elif 'chrom_gp_gamma' in p:
            labels.append(add+r'$\gamma_{\rm{Chr}}$')
        elif 'chrom_gp_log10_A' in p:
            labels.append(add+r'$\log_{10}A_{\rm{Chr}}$')
        elif 'chrom_gp_idx' in p:
            labels.append(add+r'$\chi_{\rm{Chr}}$')
        elif 'hf_red_noise_gamma' in p:
            labels.append(add+r'$\gamma_{\rm{HF}}$')
        elif 'hf_red_noise_log10_A' in p:
            labels.append(add+r'$\log_{10}A_{\rm{HF}}$')
        elif 'red_noise_gamma' in p:
            labels.append(add+r'$\gamma_{\rm{RN}}$')
        elif 'red_noise_log10_A' in p:
            labels.append(add+r'$\log_{10}A_{\rm{RN}}$')
        elif p == 'crn_log10_A':
            labels.append(r'$\log_{10}A_{\rm{CRN}}$')
        elif p == 'crn_gamma':
            labels.append(r'$\gamma_{\rm{CRN}}$')
        elif 'sw_gp_log10_sigma_ne' in p:
            labels.append(r'$\log_{10}\sigma_{n_E}$')
        elif 'exp1_log10_Amp' in p:
            labels.append(add+r'$\log_{10}A_{\rm{exp}}$')
        elif 'exp1_log10_tau' in p:
            labels.append(add+r'$\log_{10}\tau_{\rm{exp}}$')
        elif 'exp1_t0' in p:
            labels.append(add+r'$t_{\rm{exp}}$')
        elif 'exp1_idx' in p:
            labels.append(add+r'$\chi_{\rm{exp}}$')
        else:
            labels.append(p)
    return labels

In [ ]:
project_path = '/vast/palmer/home.grace/bbl29/IPTA_DR2_analysis'
figsave_dir = f'{project_path}/figs/dataset_comparisons/noise'
full_dataset = f'{project_path}/data/full_ePSRs'
lite_dataset = f'{project_path}/data/lite_unfiltered_53_ePSRs'
full_coredir = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/dr2full/advnoise'
lite_coredir = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/dr2lite_unfiltered/advnoise'
full_noisedict_path = f'{project_path}/noisedicts/dr2full_advnoise.json'
lite_noisedict_path = f'{project_path}/noisedicts/dr2lite_unfiltered_advnoise.json'
full_pardir = f'{project_path}/data/full_partim_hotfix_sw'
lite_pardir = f'{project_path}/data/lite_unfiltered_partim_final_hotfix_sw'

In [ ]:
with open(full_noisedict_path,'r') as f:
    full_noise_params = json.load(f)
with open(lite_noisedict_path,'r') as f:
    lite_noise_params = json.load(f)

In [ ]:
full_psrs = []
psr_paths = np.sort(glob.glob(f'{full_dataset}/J*'))
for psr_path in psr_paths:
    with open(psr_path,'rb') as f:
        full_psrs.append(FilePulsar(f))
full_psrnames = [psr.name for psr in full_psrs]
lite_psrs = []
psr_paths = np.sort(glob.glob(f'{lite_dataset}/J*'))
for psr_path in psr_paths:
    with open(psr_path,'rb') as f:
        lite_psrs.append(pickle.load(f))
lite_psrnames = [psr.name for psr in lite_psrs]

In [ ]:
# drop 4 pulsars with no multi-PTA measurements in Full
drop_psrnames = [lite_psrnames[i] for i, psr in enumerate(full_psrs) if len(np.unique(psr.flags['pta']))==1]
drop_psrnames, len(drop_psrnames), len(lite_psrnames) - len(drop_psrnames)

In [ ]:
# load cores
full_cores = {}
lite_cores = {}
for psrname in full_psrnames:
    full_cores[psrname] = co.Core(corepath=f'{full_coredir}/{psrname}/core.h5', burn=0)
for psrname in lite_psrnames:
    lite_cores[psrname] = co.Core(corepath=f'{lite_coredir}/{psrname}/core.h5', burn=0)

## Compare data cadence

In [ ]:
cadences = {}
for psrname in lite_psrnames:
    lp = [p for p in lite_psrs if p.name == psrname][0]
    fp = [p for p in full_psrs if p.name == psrname][0]
    dt_lite = lp.toas[1:]-lp.toas[:-1]
    dt_full = fp.toas[1:]-fp.toas[:-1]
    # To avoid overweighting the subbanded NANOGrav data, the cadence must be at least half a day
    dt_lite = dt_lite[dt_lite/const.day > 0.5]
    dt_full = dt_full[dt_full/const.day > 0.5]
    cadences[psrname] = {'lite':const.day/np.mean(dt_lite)}
    cadences[psrname]['full'] = const.day/np.mean(dt_full)
    cadences[psrname]['factor'] = cadences[psrname]['full']/cadences[psrname]['lite']
for psrname in cadences:
    print(psrname, cadences[psrname]['factor'])

In [ ]:
mean_BWs = {}
for psrname in lite_psrnames:
    mean_BWs[psrname] = {}
    lp = [p for p in lite_psrs if p.name == psrname][0]
    fp = [p for p in full_psrs if p.name == psrname][0]
    # lite
    dts = np.arange(np.min(lp.toas), np.max(lp.toas), 14*const.day)
    BWs = []
    for i in range(len(dts)-1):
        try:
            max_nu = np.max(lp.freqs[(lp.toas <= dts[i+1])*(lp.toas >= dts[i])])
            min_nu = np.min(lp.freqs[(lp.toas <= dts[i+1])*(lp.toas >= dts[i])])
            BWs.append(max_nu/min_nu)
        except:
            pass
    mean_BWs[psrname]['lite'] = np.mean(BWs)
    # full
    dts = np.arange(np.min(fp.toas), np.max(fp.toas), 14*const.day)
    BWs = []
    for i in range(len(dts)-1):
        try:
            max_nu = np.max(fp.freqs[(fp.toas <= dts[i+1])*(fp.toas >= dts[i])])
            min_nu = np.min(fp.freqs[(fp.toas <= dts[i+1])*(fp.toas >= dts[i])])
            BWs.append(max_nu/min_nu)
        except:
            pass
    mean_BWs[psrname]['full'] = np.mean(BWs)
#mean_BWs
for psrname in mean_BWs:
    print(psrname, mean_BWs[psrname]['full']/mean_BWs[psrname]['lite'])

## Compare white noise

In [ ]:
for psrname in ['J1713+0747']:#lite_psrnames:
    params = [p for p in lite_cores[psrname].params if 'efac' in p or 'equad' in p or 'ecorr' in p]
    full_Tspan = [(psr.toas.max() - psr.toas.min())/const.yr for psr in full_psrs if psr.name == psrname][0]
    lite_Tspan = [(psr.toas.max() - psr.toas.min())/const.yr for psr in lite_psrs if psr.name == psrname][0]
    psr = [p for p in lite_psrs if p.name == psrname][0]
    pta = psr.flags['pta'][0]
    ncols = 4
    if pta == "NANOGrav":
        ncols = 6
    dg.plot_chains([full_cores[psrname], lite_cores[psrname]], ncols=ncols, pars=params,
                   legend_labels=[f'DR2 \nTspan={full_Tspan:2.1f} yr',
                                  f'DR2lite ({pta}) \nTspan={lite_Tspan:2.1f} yr'], 
                   save=f'{figsave_dir}/{psrname}_wn.png')

Another thing we can try is to check every unique hite noise parameter, and make a box plot of how those parameters change

In [ ]:
wn_params = []
for psrname in lite_cores:
    wn_params.extend([p.replace(f'{psrname}_','') for p in lite_cores[psrname].params
                      if 'efac' in p or 'equad' in p or 'ecorr' in p]) 
wn_params = np.unique(wn_params)
#
wn_psrs = {}
for p in wn_params:
    psr_list = []
    for psrname in lite_cores:
        if f'{psrname}_{p}' in lite_cores[psrname].params:
            psr_list.append(psrname)
    wn_psrs[p] = psr_list
wn_psrs

In [ ]:
len(wn_psrs)/6

In [ ]:
# Do the distributions overlap?
fig, axes = plt.subplots(14,6, figsize=(6*2,14*2))
axes = np.array(axes).flatten()
for i, param in enumerate(wn_psrs):
    if 'efac' in param:
        color = 'C0'
    if 'equad' in param:
        color = 'C1'
    if 'ecorr' in param:
        color = 'C2'
    for j, psrname in enumerate(wn_psrs[param]):
        param_name = f'{psrname}_{param}'
        axes[i].hist(lite_cores[psrname](param_name), density=True, bins=30, color='k', histtype='step')
        axes[i].hist(full_cores[psrname](param_name), density=True, bins=30, color=color, histtype='step')
    axes[i].set_xlabel(param)
    axes[i].set_yticks([])
fig.tight_layout()

In [ ]:
# What about differences in Bayes factors?
eq_BF_ratios = {}
ec_BF_ratios = {}
for i, param in enumerate(wn_psrs):
    for j, psrname in enumerate(wn_psrs[param]):
        param_name = f'{psrname}_{param}'
        lite_BF = gorilla_bf(lite_cores[psrname](param_name), max=-4, min=-10)
        full_BF = gorilla_bf(full_cores[psrname](param_name), max=-4, min=-10)
        if np.isnan(lite_BF) and np.isnan(full_BF):
            BF_ratio = np.nan
        elif np.isnan(lite_BF):
            lite_BF = np.inf
        elif np.isnan(full_BF):
            full_BF = np.inf
        else:
            BF_ratio = full_BF/lite_BF
        if 'equad' in param:
            eq_BF_ratios[param_name] = BF_ratio
        elif 'ecorr' in param:
            ec_BF_ratios[param_name] = BF_ratio
#eq_BF_ratios, ec_BF_ratios

In [ ]:
for param in eq_BF_ratios:
    if eq_BF_ratios[param] > 10 or eq_BF_ratios[param] < 0.01:
        print(param, eq_BF_ratios[param])

In [ ]:
for param in ec_BF_ratios:
    if ec_BF_ratios[param] > 10 or ec_BF_ratios[param] < 0.01:
        print(param, ec_BF_ratios[param])

### Get White noise RMS across same set of TOAs

In [ ]:
lite_RMS_errs = {}
full_RMS_errs = {}
full_RMS_all_errs = {}
n_realizations = 3000
for psrname in lite_psrnames:
    print(psrname)
    # test getting the N matrix
    lite_RMS_errs[psrname] = np.zeros(n_realizations)
    full_RMS_errs[psrname] = np.zeros(n_realizations)
    full_RMS_all_errs[psrname] = np.zeros(n_realizations)
    lite_psr = [psr for psr in lite_psrs if psr.name == psrname][0]
    full_psr = [psr for psr in full_psrs if psr.name == psrname][0]
    lite_pta = adv_noise_block(lite_psr, full_pta_analysis=False, dataset='lite', psr_model=False,
                               tm_marg=False, tm_svd=False, gp_ecorr=True)
    full_pta = adv_noise_block(full_psr, full_pta_analysis=False, dataset='full', psr_model=False,
                               tm_marg=False, tm_svd=False, gp_ecorr=True)
    for i in tqdm.tqdm(range(n_realizations)):
        # lite - use all data
        rand_idx = np.random.choice(np.arange(len(lite_cores[psrname]('lnlike'))))
        sample = {param:lite_cores[psrname](param)[rand_idx] for param in lite_cores[psrname].params}
        Ndiag = lite_pta.get_ndiag(sample)[0]
        lite_RMS_errs[psrname][i] = np.sqrt(np.sum(Ndiag)/len(Ndiag))
        # full - apply mask to use only the same TOAs as in lite
        rand_idx = np.random.choice(np.arange(len(full_cores[psrname]('lnlike'))))
        sample = {param:full_cores[psrname](param)[rand_idx] for param in full_cores[psrname].params}
        mask = np.isin(full_psr.toas, lite_psr.toas)
        Ndiag = full_pta.get_ndiag(sample)[0]
        full_RMS_errs[psrname][i] = np.sqrt(np.sum(Ndiag[mask])/len(Ndiag[mask]))
        full_RMS_all_errs[psrname][i] = np.sqrt(np.sum(Ndiag)/len(Ndiag))
        # noiseless - assume no efac equad, or ecorr
        for param in lite_cores[psrname].params:
            if 'efac' in param:
                sample[param] = 1
            elif 'equad' in param or 'ecorr' in param:
                sample[param] = -np.inf
        Ndiag = lite_pta.get_ndiag(sample)[0]

In [ ]:
noiseless_RMS_errs = {}
for psrname in lite_psrnames:
    lite_psr = [psr for psr in lite_psrs if psr.name == psrname][0]
    lite_pta = adv_noise_block(lite_psr, full_pta_analysis=False, dataset='lite', psr_model=False,
                               tm_marg=False, tm_svd=False, gp_ecorr=True)
    rand_idx = np.random.choice(np.arange(len(lite_cores[psrname]('lnlike'))))
    sample = {param:lite_cores[psrname](param)[rand_idx] for param in lite_cores[psrname].params}
    for param in lite_cores[psrname].params:
        if 'efac' in param:
            sample[param] = 1
        elif 'equad' in param or 'ecorr' in param:
            sample[param] = -np.inf
    Ndiag = lite_pta.get_ndiag(sample)[0]
    noiseless_RMS_errs[psrname] = np.sqrt(np.sum(Ndiag)/len(Ndiag))

In [ ]:
53/6

In [ ]:
fig, axes = plt.subplots(3,6,figsize=(6*2, 3*2))
axes = np.array(axes).flatten()
for i, psrname in enumerate(lite_psrnames):
    axes[i].hist(lite_RMS_errs[psrname]*1e6, color='C1', density=True, bins=30, histtype='step')
    axes[i].hist(full_RMS_errs[psrname]*1e6, color='C0', density=True, bins=30, histtype='step')
    axes[i].set_xlabel(r'White noise RMS ($\mu$s)')
    axes[i].set_title(f'PSR {psrname}')
    axes[i].axvline(noiseless_RMS_errs[psrname]*1e6, color='k', ls='--')
handles = [mlines.Line2D([],[],color='C0',label='Full DR2')]
handles += [mlines.Line2D([],[],color='C1',label='DR2 Lite')]
handles += [mlines.Line2D([],[],color='k',ls='--',label='Original TOA error')]
fig.legend(handles=handles, loc='lower right', bbox_to_anchor=(1, 0.15))
fig.delaxes(axes[-1])
fig.tight_layout()
fig.savefig(f'{figsave_dir}/RMS_same_toas.png', bbox_inches='tight', dpi=400)
plt.show()

### Get White noise RMS across each dataset

In [ ]:
full_noiseless_RMS_errs = {}
for psrname in lite_psrnames:
    full_psr = [psr for psr in full_psrs if psr.name == psrname][0]
    full_pta = adv_noise_block(full_psr, full_pta_analysis=False, dataset='lite', psr_model=False,
                               tm_marg=False, tm_svd=False, gp_ecorr=True)
    rand_idx = np.random.choice(np.arange(len(full_cores[psrname]('lnlike'))))
    sample = {param:full_cores[psrname](param)[rand_idx] for param in full_cores[psrname].params}
    for param in full_cores[psrname].params:
        if 'efac' in param:
            sample[param] = 1
        elif 'equad' in param or 'ecorr' in param:
            sample[param] = -np.inf
    Ndiag = full_pta.get_ndiag(sample)[0]
    full_noiseless_RMS_errs[psrname] = np.sqrt(np.sum(Ndiag)/len(Ndiag))

In [ ]:
fig, axes = plt.subplots(3,6,figsize=(6*2, 3*2))
axes = np.array(axes).flatten()
for i, psrname in enumerate(lite_psrnames):
    axes[i].hist(lite_RMS_errs[psrname]*1e6, color='C1', density=True, bins=30, histtype='step')
    axes[i].hist(full_RMS_all_errs[psrname]*1e6, color='C0', density=True, bins=30, histtype='step')
    axes[i].set_xlabel(r'White noise RMS ($\mu$s)')
    axes[i].set_title(f'PSR {psrname}')
    axes[i].axvline(noiseless_RMS_errs[psrname]*1e6, color='C1', ls='--')
    axes[i].axvline(full_noiseless_RMS_errs[psrname]*1e6, color='C0', ls='--')
handles = [mlines.Line2D([],[],color='C0',label='Full DR2')]
handles += [mlines.Line2D([],[],color='C1',label='DR2 Lite')]
handles += [mlines.Line2D([],[],color='C0',ls='--',label='Full DR2 TOA error')]
handles += [mlines.Line2D([],[],color='C1',ls='--',label='DR2 Lite TOA error')]
fig.legend(handles=handles, loc='lower right', bbox_to_anchor=(1, 0.15))
fig.delaxes(axes[-1])
fig.tight_layout()
fig.savefig(f'{figsave_dir}/RMS_all_toas.png', bbox_inches='tight', dpi=400)
plt.show()

## Make table of signal significances

We are interested not in what the noise parameters are, but whether we detect them at all!

In [ ]:
table = []
for psrname in lite_psrnames:
    if psrname in drop_psrnames:
        continue
    line = []
    line.append(psrname)
    if psrname == 'J1012+5307':
        params = [f'{psrname}_hf_red_noise_log10_A', f'{psrname}_dm_gp_log10_A']
    else:
        params = [f'{psrname}_red_noise_log10_A', f'{psrname}_dm_gp_log10_A']
    if f'{psrname}_chrom_gp_log10_A' in lite_cores[psrname].params:
        params += [f'{psrname}_chrom_gp_log10_A']
    else:
        params += [None]
    if f'{psrname}_sw_gp_log10_sigma_ne' in lite_cores[psrname].params:
        params += [f'{psrname}_sw_gp_log10_sigma_ne']
    else:
        params += [None]
    for core in [lite_cores[psrname], full_cores[psrname]]:
        # red noise first
        for param in params:
            #bf_conds = [psrname == 'J0613-0200' and core == lite_cores[psrname] and 'dm_gp' in params]
            if param is None:
                line.append('$-$')
            else:
                if 'sw_gp' in param:
                    BF = gorilla_bf(core(param), max=2, min=-4)
                else:
                    BF = gorilla_bf(core(param), max=-11, min=-20)
                if np.isnan(BF):
                    line.append(r'$>\mathbf{10^3}$')
                elif BF > 10:# or np.any(bf_conds):
                    #if np.any(bf_conds):
                    #    print(psrname)
                    line.append(fr'$\mathbf{{{BF:0.1f}}}$')
                else:
                    line.append(fr'${BF:0.1f}$')
    table.append(' & '.join(line))
print(' \\\ \n'.join(table))

In [ ]:
all_pars = []
for psrname in lite_psrnames:
    pars = [p.replace(f'{psrname}_','') for p in lite_cores[psrname].params if not 'efac' in p
            and not 'equad' in p and not 'ecorr' in p]
    all_pars.extend(pars)
np.unique(all_pars)

## Compare noise corners

In [ ]:
for psrname in lite_psrnames:
    if psrname in drop_psrnames:
        continue
    params = [p for p in lite_cores[psrname].params if 'gp' in p or 'red_noise' in p or 'exp' in p]
    labels = make_labels(params, psrname, add_psrname=False)
    # full
    full_Tspan = [(psr.toas.max() - psr.toas.min())/const.yr for psr in full_psrs if psr.name == psrname][0]
    p_idxs = [full_cores[psrname].params.index(p) for p in params]
    fig = corner.corner(full_cores[psrname].chain[:,p_idxs], levels=(0.68,0.95), labels=labels,
                        color='k', plot_density=False, plot_datapoints=False,
                        no_fill_contours=True, hist_kwargs={'density':True},
                        label_kwargs={'fontsize':20})
    # lite
    lite_Tspan = [(psr.toas.max() - psr.toas.min())/const.yr for psr in lite_psrs if psr.name == psrname][0]
    pta = [np.unique(psr.flags['pta'])[0] for psr in lite_psrs if psr.name == psrname][0]
    p_idxs = [lite_cores[psrname].params.index(p) for p in params]
    fig = corner.corner(lite_cores[psrname].chain[:,p_idxs], levels=(0.68,0.95), fig=fig,
                        color='C0', plot_density=False, plot_datapoints=True,
                        no_fill_contours=True, hist_kwargs={'density':True},
                        label_kwargs={'fontsize':20})
    lines = [mlines.Line2D([],[],color='k',label=f'DR2 \nTspan={full_Tspan:2.1f} yr')]
    lines += [mlines.Line2D([],[],color='C0',label=f'DR2lite ({pta}) \nTspan={lite_Tspan:2.1f} yr')]
    fig.legend(handles=lines, fontsize=20)
    fig.suptitle(f'PSR {psrname}', fontsize=30, x=0.3, y=1.01)
    fig.savefig(f'{figsave_dir}/{psrname}_corner.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
for psrname in lite_psrnames:
    fig, ax = plt.subplots(figsize=(8,3))
    psr1 = [p for p in full_psrs if p.name == psrname][0]
    psr2 = [p for p in lite_psrs if p.name == psrname][0]
    pta = psr2.flags['pta'][0]
    mask = psr1.flags['pta'] == pta
    ax.errorbar(psr1.toas[mask]/const.day, 1e6*psr1.residuals[mask], yerr=1e6*psr1.toaerrs[mask],
                ls='none', fmt='.C0')
    ax.errorbar(psr2.toas/const.day, 1e6*psr2.residuals, yerr=1e6*psr2.toaerrs, ls='none', fmt='.C1')
    ax.set_title(psrname)
    plt.show()

In [ ]:
corepath_crn = f'/vast/palmer/home.grace/bbl29/project/IPTA_DR2_analysis/dr2full/CRN13_advnoise/core.h5'
core_crn = co.Core(corepath=corepath_crn)

In [ ]:
for psrname in ['J0613-0200', 'J1909-3744', 'J1910+1256']:#lite_psrnames:
    if psrname in drop_psrnames:
        continue
    params = [p for p in lite_cores[psrname].params if ('gp' in p and not 'idx' in p) or 'red_noise' in p]
    # or 'exp' in p]
    labels = make_labels(params, psrname, add_psrname=False)
    ndim = len(params)
    # lite
    lite_Tspan = [(psr.toas.max() - psr.toas.min())/const.yr for psr in lite_psrs if psr.name == psrname][0]
    pta = [np.unique(psr.flags['pta'])[0] for psr in lite_psrs if psr.name == psrname][0]
    p_idxs = [lite_cores[psrname].params.index(p) for p in params]
    fig = corner.corner(lite_cores[psrname].chain[:,p_idxs], levels=(0.68,0.95), labels=labels, bins=20,
                        color='C1', plot_density=False, plot_datapoints=False, smooth=1,
                        fill_contours=True, no_fill_contours=True, hist_kwargs={'density':True},
                        label_kwargs={'fontsize':30},
                        contourf_kwargs={'colors': [(*mpl.colors.to_rgb(default_colors[1]), 0),
                                                    (*mpl.colors.to_rgb(default_colors[1]), 0.25),
                                                    (*mpl.colors.to_rgb(default_colors[1]), 0.5)]})
    # full
    full_Tspan = [(psr.toas.max() - psr.toas.min())/const.yr for psr in full_psrs if psr.name == psrname][0]
    p_idxs = [full_cores[psrname].params.index(p) for p in params]
    fig = corner.corner(full_cores[psrname].chain[:,p_idxs], levels=(0.68,0.95), fig=fig, bins=20,
                        color='C0', plot_density=False, plot_datapoints=False, smooth=1,
                        fill_contours=True, no_fill_contours=True, hist_kwargs={'density':True},
                        contourf_kwargs={'colors': [(*mpl.colors.to_rgb(default_colors[0]), 0),
                                                    (*mpl.colors.to_rgb(default_colors[0]), 0.25),
                                                    (*mpl.colors.to_rgb(default_colors[0]), 0.5)]})
    # EDR2 CRN
    fig.subplots_adjust(right=1.5,top=1.5)
    axes = np.array(fig.axes).reshape((ndim,ndim))
    for ax1 in axes:
        for ax in ax1:
            ax.tick_params(axis='both', labelsize=20)
    ax_idxs = [params.index(p) for p in params if f'{psrname}_red_noise' in p]
    p_idxs = [core_crn.params.index(p) for p in ['crn_gamma', 'crn_log10_A']]
    for ax_idx, p_idx in zip(ax_idxs, p_idxs):
        ax = axes[ax_idx,ax_idx]
        ax.hist(core_crn.chain[:,p_idx], density=True, color='k', histtype='step', bins=20)
        y, _ = np.histogram(core_crn.chain[:,p_idx], density=True, bins=20)
        ax.set_ylim([0,1.1*np.max(y)])
    ax = axes[ax_idxs[1],ax_idxs[0]]
    xlabels = ax.get_xticklabels()
    ylabels = ax.get_yticklabels()
    corner.hist2d(core_crn('crn_gamma'), core_crn('crn_log10_A'), ax=ax, levels=(0.68,0.95), bins=20,
                  color='k', plot_density=False, plot_datapoints=False, smooth=1, new_fig=False, #force_range=True,
                  fill_contours=True, no_fill_contours=True, label_kwargs={'fontsize':20},
                  contourf_kwargs={'colors': [(0, 0, 0, 0), (0, 0, 0, 0.25), (0, 0, 0, 0.5)]})
    ax.set_xticklabels(xlabels)
    ax.set_yticklabels(ylabels)
    lines = [mlines.Line2D([],[],color='C0',label=f'Full DR2 ({psrname.replace("-","$-$")}) '
                           f'\nTspan={full_Tspan:2.1f} yr')]
    lines += [mlines.Line2D([],[],color='C1',label=f'DR2 Lite ({psrname.replace("-","$-$")}) '
                            f'\nTspan={lite_Tspan:2.1f} yr')]
    lines += [mlines.Line2D([],[],color='k',label=f'Full DR2 CRN')]
    if psrname == 'J0613-0200':
        fig.legend(handles=lines, fontsize=45, frameon=False)
        #fig.suptitle(f'PSR {psrname.replace("-",r"$-$")}', fontsize=43, x=0.3, y=1.01)
    else:
        fig.legend(handles=lines, fontsize=30, frameon=False)
        #fig.suptitle(f'PSR {psrname.replace("-",r"$-$")}', fontsize=30, x=0.3, y=1.01)
    fig.savefig(f'{figsave_dir}/{psrname}_corner2.png', dpi=300, pad_inches=0.3, bbox_inches='tight')
    plt.show()

In [ ]:
J1955+2908

In [ ]:
psrname = 'J0613-0200'
core = lite_cores[psrname]
mask = core(f'{psrname}_chrom_gp_log10_A') < -18
DMBF = gorilla_bf(core(f'{psrname}_dm_gp_log10_A')[mask], min=-20, max=-11)
RNBF = gorilla_bf(core(f'{psrname}_red_noise_log10_A')[mask], min=-20, max=-11)
print(DMBF, RNBF)
mask = core(f'{psrname}_dm_gp_log10_A') < -18
ChrBF = gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11)
RNBF = gorilla_bf(core(f'{psrname}_red_noise_log10_A')[mask], min=-20, max=-11)
print(ChrBF, RNBF)
mask = core(f'{psrname}_red_noise_log10_A') < -18
ChrBF = gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11)
DMBF = gorilla_bf(core(f'{psrname}_dm_gp_log10_A')[mask], min=-20, max=-11)
print(ChrBF, DMBF)

In [ ]:
psrname = 'J1022+1001'
core = lite_cores[psrname]
mask = core(f'{psrname}_dm_gp_log10_A') < -18
RNBF = gorilla_bf(core(f'{psrname}_red_noise_log10_A')[mask], min=-20, max=-11)
print(RNBF)

In [ ]:
psrname = 'J1024-0719'
core = lite_cores[psrname]
mask = core(f'{psrname}_dm_gp_log10_A') < -18
RNBF = gorilla_bf(core(f'{psrname}_red_noise_log10_A')[mask], min=-20, max=-11)
print(RNBF)

In [ ]:
psrname = 'J1455-3330'
core = lite_cores[psrname]
mask = core(f'{psrname}_red_noise_log10_A') < -18
DMBF = gorilla_bf(core(f'{psrname}_dm_gp_log10_A')[mask], min=-20, max=-11)
print(DMBF)

In [ ]:
psrname = 'J1600-3053'
core = lite_cores[psrname]
mask = core(f'{psrname}_chrom_gp_log10_A') < -18
#ChrBF = gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11)
print(gorilla_bf(core(f'{psrname}_dm_gp_log10_A')[mask], min=-20, max=-11))
#print(gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11))
print(gorilla_bf(core(f'{psrname}_red_noise_log10_A')[mask], min=-20, max=11))
print(gorilla_bf(core(f'{psrname}_sw_gp_log10_sigma_ne')[mask], min=-4, max=2))

In [ ]:
psrname = 'J1600-3053'
core = lite_cores[psrname]
mask = (core(f'{psrname}_red_noise_log10_A') < -18) * (core(f'{psrname}_dm_gp_log10_A') < -18)
#ChrBF = gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11)
print(gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11))
print(gorilla_bf(core(f'{psrname}_sw_gp_log10_sigma_ne')[mask], min=-4, max=2))

In [ ]:
psrname = 'J1643-1224'
core = lite_cores[psrname]
mask = core(f'{psrname}_chrom_gp_log10_A') < -18
#ChrBF = gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11)
print(gorilla_bf(core(f'{psrname}_dm_gp_log10_A')[mask], min=-20, max=-11))
print(gorilla_bf(core(f'{psrname}_red_noise_log10_A')[mask], min=-20, max=-11))

In [ ]:
psrname = 'J1738+0333'
core = full_cores[psrname]
mask = core(f'{psrname}_red_noise_log10_A') < -18
DMBF = gorilla_bf(core(f'{psrname}_dm_gp_log10_A')[mask], min=-20, max=-11)
print(DMBF)

In [ ]:
psrname = 'J1744-1134'
core = lite_cores[psrname]
mask = core(f'{psrname}_red_noise_log10_A') < -18
DMBF = gorilla_bf(core(f'{psrname}_dm_gp_log10_A')[mask], min=-20, max=-11)
print(DMBF)

In [ ]:
psrname = 'J1918-0642'
core = lite_cores[psrname]
mask = core(f'{psrname}_sw_gp_log10_sigma_ne') < -2
#ChrBF = gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11)
print(gorilla_bf(core(f'{psrname}_dm_gp_log10_A')[mask], min=-20, max=-11))
print(gorilla_bf(core(f'{psrname}_red_noise_log10_A')[mask], min=-20, max=-11))
mask = core(f'{psrname}_dm_gp_log10_A') < -18
#ChrBF = gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11)
print(gorilla_bf(core(f'{psrname}_sw_gp_log10_sigma_ne')[mask], min=-4, max=2))
print(gorilla_bf(core(f'{psrname}_red_noise_log10_A')[mask], min=-20, max=-11))

In [ ]:
psrname = 'J2145-0750'
core = lite_cores[psrname]
mask = core(f'{psrname}_sw_gp_log10_sigma_ne') < -2
#ChrBF = gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11)
print(gorilla_bf(core(f'{psrname}_dm_gp_log10_A')[mask], min=-20, max=-11))
print(gorilla_bf(core(f'{psrname}_red_noise_log10_A')[mask], min=-20, max=-11))
mask = core(f'{psrname}_dm_gp_log10_A') < -18
#ChrBF = gorilla_bf(core(f'{psrname}_chrom_gp_log10_A')[mask], min=-20, max=-11)
print(gorilla_bf(core(f'{psrname}_sw_gp_log10_sigma_ne')[mask], min=-4, max=2))
print(gorilla_bf(core(f'{psrname}_red_noise_log10_A')[mask], min=-20, max=-11))

In [ ]:
np.count_nonzero(mask)

In [ ]:
psrname = 'J1713+0747'
params = [p for p in lite_cores[psrname].params if 'red_noise' in p or 'dm_gp' in p]
labels = make_labels(params, psrname, add_psrname=False)
pta = [np.unique(psr.flags['pta'])[0] for psr in lite_psrs if psr.name == psrname][0]
p_idxs = [lite_cores[psrname].params.index(p) for p in params]
mask = lite_cores[psrname](f'{psrname}_chrom_gp_log10_A') < -18
fig = corner.corner(lite_cores[psrname].chain[:,p_idxs][mask], levels=(0.68,0.95), labels=labels, bins=20,
                    color='C1', plot_density=False, plot_datapoints=False, smooth=1,
                    fill_contours=True, no_fill_contours=True, hist_kwargs={'density':True},
                    label_kwargs={'fontsize':20},
                    contourf_kwargs={'colors': [(*mpl.colors.to_rgb(default_colors[1]), 0),
                                                (*mpl.colors.to_rgb(default_colors[1]), 0.25),
                                                (*mpl.colors.to_rgb(default_colors[1]), 0.5)]})

In [ ]:
np.count_nonzero(mask)

#### Compare detection statistics on ARN, DM, chromatic noise

In [ ]:
def get_BFr(param_name, lite_core, full_core, max=-4, min=-10):
    lite_BF = gorilla_bf(lite_core(param_name), max=max, min=min)
    full_BF = gorilla_bf(full_core(param_name), max=max, min=min)
    if np.isnan(lite_BF) and np.isnan(full_BF):
        BFr = np.nan
    else:
        if np.isnan(lite_BF):
            lite_BF = np.inf
        elif np.isnan(full_BF):
            full_BF = np.inf
        BFr = full_BF/lite_BF
    return BFr

In [ ]:
for j, psrname in enumerate(lite_psrnames):
    print(psrname)
    for i, param in enumerate(lite_cores[psrname].params):
        if param == f'{psrname}_red_noise_log10_A':
            BF_ratio = get_BFr(param, lite_cores[psrname], full_cores[psrname], max=-11, min=-20)
            if BF_ratio > 20 or np.isnan(BF_ratio) or BF_ratio < 0.5:
                print(param, BF_ratio)
        if param == f'{psrname}_hf_red_noise_log10_A':
            BF_ratio = get_BFr(param, lite_cores[psrname], full_cores[psrname], max=-11, min=-20)
            if BF_ratio > 20 or np.isnan(BF_ratio) or BF_ratio < 0.5:
                print(param, BF_ratio)
        if param == f'{psrname}_dm_gp_log10_A':
            BF_ratio = get_BFr(param, lite_cores[psrname], full_cores[psrname], max=-11, min=-20)
            if BF_ratio > 20 or np.isnan(BF_ratio) or BF_ratio < 0.5:
                print(param, BF_ratio)
        if param == f'{psrname}_chrom_gp_log10_A':
            BF_ratio = get_BFr(param, lite_cores[psrname], full_cores[psrname], max=-11, min=-20)
            if BF_ratio > 20 or np.isnan(BF_ratio) or BF_ratio < 0.5:
                print(param, BF_ratio)
        if param == f'{psrname}_sw_gp_log10_sigma_ne':
            BF_ratio = get_BFr(param, lite_cores[psrname], full_cores[psrname], max=-11, min=-20)
            if BF_ratio > 20 or np.isnan(BF_ratio) or BF_ratio < 0.5:
                print(param, BF_ratio)
    print()

### J1909: What if we use uniform RN priors?

In [ ]:
from targeted_cws_ng15.utils import get_prior_log_weights

In [ ]:
lite_cores[psrname].priors

In [ ]:
psrname = 'J1909-3744'
lnweight = get_prior_log_weights(lite_cores[psrname](f'{psrname}_red_noise_log10_A'),
                                 prior1_args=[-20,-11],prior0_args=[-20,-11])

In [ ]:
psrname = 'J1909-3744'
params = [p for p in lite_cores[psrname].params if ('gp' in p and not 'idx' in p) or 'red_noise' in p]
# or 'exp' in p]
labels = make_labels(params, psrname, add_psrname=False)
ndim = len(params)
# lite
lite_Tspan = [(psr.toas.max() - psr.toas.min())/const.yr for psr in lite_psrs if psr.name == psrname][0]
pta = [np.unique(psr.flags['pta'])[0] for psr in lite_psrs if psr.name == psrname][0]
p_idxs = [lite_cores[psrname].params.index(p) for p in params]
fig = corner.corner(lite_cores[psrname].chain[:,p_idxs], levels=(0.68,0.95), labels=labels, bins=20,
                    color='C1', plot_density=False, plot_datapoints=False, smooth=1,
                    fill_contours=True, no_fill_contours=True, hist_kwargs={'density':True},
                    label_kwargs={'fontsize':20}, weights=np.exp(lnweight),
                    contourf_kwargs={'colors': [(*mpl.colors.to_rgb(default_colors[1]), 0),
                                                (*mpl.colors.to_rgb(default_colors[1]), 0.25),
                                                (*mpl.colors.to_rgb(default_colors[1]), 0.5)]})
# full
full_Tspan = [(psr.toas.max() - psr.toas.min())/const.yr for psr in full_psrs if psr.name == psrname][0]
p_idxs = [full_cores[psrname].params.index(p) for p in params]
fig = corner.corner(full_cores[psrname].chain[:,p_idxs], levels=(0.68,0.95), fig=fig, bins=20,
                    color='C0', plot_density=False, plot_datapoints=False, smooth=1,
                    fill_contours=True, no_fill_contours=True, hist_kwargs={'density':True},
                    label_kwargs={'fontsize':20},
                    contourf_kwargs={'colors': [(*mpl.colors.to_rgb(default_colors[0]), 0),
                                                (*mpl.colors.to_rgb(default_colors[0]), 0.25),
                                                (*mpl.colors.to_rgb(default_colors[0]), 0.5)]})
# EDR2 CRN
axes = np.array(fig.axes).reshape((ndim,ndim))
ax_idxs = [params.index(p) for p in params if f'{psrname}_red_noise' in p]
p_idxs = [core_crn.params.index(p) for p in ['crn_gamma', 'crn_log10_A']]
for ax_idx, p_idx in zip(ax_idxs, p_idxs):
    ax = axes[ax_idx,ax_idx]
    ax.hist(core_crn.chain[:,p_idx], density=True, color='k', histtype='step', bins=20)
    y, _ = np.histogram(core_crn.chain[:,p_idx], density=True, bins=20)
    ax.set_ylim([0,1.1*np.max(y)])
ax = axes[ax_idxs[1],ax_idxs[0]]
xlabels = ax.get_xticklabels()
ylabels = ax.get_yticklabels()
corner.hist2d(core_crn('crn_gamma'), core_crn('crn_log10_A'), ax=ax, levels=(0.68,0.95), bins=20,
              color='k', plot_density=False, plot_datapoints=False, smooth=1, new_fig=False, #force_range=True,
              fill_contours=True, no_fill_contours=True, label_kwargs={'fontsize':20},
              contourf_kwargs={'colors': [(0, 0, 0, 0), (0, 0, 0, 0.25), (0, 0, 0, 0.5)]})
ax.set_xticklabels(xlabels)
ax.set_yticklabels(ylabels)
lines = [mlines.Line2D([],[],color='C0',label=f'Full DR2 \nTspan={full_Tspan:2.1f} yr')]
lines += [mlines.Line2D([],[],color='C1',label=f'DR2 Lite ({pta}) \nTspan={lite_Tspan:2.1f} yr')]
lines += [mlines.Line2D([],[],color='k',label=f'Full DR2 CRN')]
fig.legend(handles=lines, fontsize=20)
fig.suptitle(f'PSR {psrname}', fontsize=30, x=0.3, y=1.01)
fig.savefig(f'{figsave_dir}/{psrname}_corner2_UL.png', dpi=300, bbox_inches='tight')
plt.show()

## Compare GP spectra

Just red noise. Which pulsars show excursions at 4 nHz?

In [ ]:
full_pta_Tspan=952746385.6296968

In [ ]:
from enterprise.signals.utils import ConditionalGP

In [ ]:
n_realizations = 500
continue_ = True
for psrname in lite_cores:
    if psrname == 'J1911-1114':
        continue_ = False
    if continue_:
        continue
    lite_psr = [p for p in lite_psrs if p.name == psrname][0]
    full_psr = [p for p in full_psrs if p.name == psrname][0]
    lite_pta = adv_noise_block(lite_psr, full_pta_analysis=False, dataset='full', psr_model=False,
                               tm_marg=False, tm_svd=False)
    full_pta = adv_noise_block(full_psr, full_pta_analysis=False, dataset='full', psr_model=False,
                               tm_marg=False, tm_svd=False)
    pta_label = np.unique(lite_psr.flags['pta'])[0]
    lite_pta.set_default_params(lite_noise_params)
    full_pta.set_default_params(full_noise_params)
    for signal in lite_pta.signals:
        lite_pta.signals[signal].basis_combine = False
    for signal in full_pta.signals:
        full_pta.signals[signal].basis_combine = False
    gp_lite = ConditionalGP(lite_pta)
    gp_full = ConditionalGP(full_pta)

    lite_Tspan = lite_psr.toas.max() - lite_psr.toas.min()
    lite_Nf = int(np.ceil(lite_Tspan*30/full_pta_Tspan))
    lite_freqs = np.linspace(1/lite_Tspan, lite_Nf/lite_Tspan, lite_Nf)
    full_Tspan = full_psr.toas.max() - full_psr.toas.min()
    full_Nf = int(np.ceil(full_Tspan*30/full_pta_Tspan))
    full_freqs = np.linspace(1/full_Tspan, full_Nf/full_Tspan, full_Nf)

    rho_lite = np.zeros((n_realizations,lite_Nf))
    rho_full = np.zeros((n_realizations,full_Nf))
    for i in tqdm.tqdm(range(n_realizations)):
        # lite
        try:
            idx = np.random.choice(np.arange(len(lite_cores[psrname]('lnlike'))))
            params = {p:lite_cores[psrname](p)[idx] for p in lite_cores[psrname].params}
            coefs = gp_lite.sample_coefficients(params, n=1)
            a = coefs[0][f'{psrname}_red_noise_coefficients']
            rho_lite[i] = np.sqrt(a[::2]**2 + a[1::2]**2)
            # full
            idx = np.random.choice(np.arange(len(full_cores[psrname]('lnlike'))))
            params = {p:full_cores[psrname](p)[idx] for p in full_cores[psrname].params}
            coefs = gp_full.sample_coefficients(params, n=1)
            a = coefs[0][f'{psrname}_red_noise_coefficients']
            rho_full[i] = np.sqrt(a[::2]**2 + a[1::2]**2)
        except LinAlgError:
            rho_lite[i] = np.ones(len(lite_freqs))*np.nan
            rho_full[i] = np.ones(len(full_freqs))*np.nan

    fig, ax = plt.subplots(figsize=(6,3))
    # Full
    med = np.nanmedian(rho_full, axis=0)
    err = np.nanquantile(rho_full, [0.16,0.84], axis=0)
    ax.plot(full_freqs*1e9, med, '.C0', ms=1)
    ax.fill_between(full_freqs*1e9, err[0], err[1], alpha=0.2, color='C0', label='DR2')
    # lite
    med = np.median(rho_lite, axis=0)
    err = np.quantile(rho_lite, [0.16,0.84], axis=0)
    ax.plot(lite_freqs*1e9, med, '.C1', ms=1)
    ax.fill_between(lite_freqs*1e9, err[0], err[1], alpha=0.2, color='C1', label=f'DR2lite ({pta_label})')
    # labels
    ax.grid(lw=0.3)
    ax.set_title(psrname)
    ax.loglog()
    ax.legend()
    ax.set_xlabel(r'$f$ (nHz)')
    ax.set_ylabel(r'$\rho$ (s)')
    ax.scatter([4], [10**(-6.3)], marker='s', color='k', s=10)
    fig.tight_layout()
    fig.subplots_adjust(hspace=0)
    fig.savefig(f'{figsave_dir}/{psrname}_rn_spectra.png', dpi=300, bbox_inches='tight')
    plt.show()

## Compare GPs

In [ ]:
lite_psrnames

In [ ]:
n_realizations = 500
for psrname in lite_psrnames[10:]:
    lite_psr = [p for p in lite_psrs if p.name == psrname][0]
    full_psr = [p for p in full_psrs if p.name == psrname][0]
    lite_pta = adv_noise_block(lite_psr, full_pta_analysis=False, dataset='lite', psr_model=False,
                               tm_marg=False, tm_svd=False)
    full_pta = adv_noise_block(full_psr, full_pta_analysis=False, dataset='full', psr_model=False,
                               tm_marg=False, tm_svd=False)
    pta_label = np.unique(lite_psr.flags['pta'])[0]
    lite_pta.set_default_params(lite_noise_params)
    full_pta.set_default_params(full_noise_params)
    for signal in lite_pta.signals:
        lite_pta.signals[signal].basis_combine = False
    for signal in full_pta.signals:
        full_pta.signals[signal].basis_combine = False
    lite_sig = ppu.make_gp_realizations(lite_cores[psrname], lite_pta, lite_psr, all_dm=True,
                                        tm_params=['Offset','F0', 'F1', 'DM', 'DM1', 'DM2', 'NE_SW'],
                                        pardir=lite_pardir, n_realizations=n_realizations,
                                        use_dm_units=True, ref_MHz=1400)
    lite_sig[f'{psrname}_red_noise'] += (lite_sig[f'{psrname}_Offset'] + lite_sig[f'{psrname}_F0'] +
                                         lite_sig[f'{psrname}_F1'])
    if f'{psrname}_hf_red_noise' in lite_sig:
        lite_sig[f'{psrname}_red_noise'] += lite_sig[f'{psrname}_hf_red_noise']
    full_sig = ppu.make_gp_realizations(full_cores[psrname], full_pta, full_psr, all_dm=True,
                                        tm_params=['Offset', 'F0', 'F1', 'DM', 'DM1', 'DM2', 'NE_SW'],
                                        pardir=full_pardir, n_realizations=n_realizations,
                                        use_dm_units=True, ref_MHz=1400)
    full_sig[f'{psrname}_red_noise'] += (full_sig[f'{psrname}_Offset'] + full_sig[f'{psrname}_F0'] +
                                         full_sig[f'{psrname}_F1'])
    if f'{psrname}_hf_red_noise' in full_sig:
        full_sig[f'{psrname}_red_noise'] += full_sig[f'{psrname}_hf_red_noise']
    # plot
    labels = [r'$\Delta$DM ($10^{-3}$ pc/cm$^3$)', r'$\Delta t_{\rm{RN}}$ ($\mu$s)']
    sig_names = [f'{psrname}_dm_gp', f'{psrname}_red_noise']
    if f'{psrname}_chrom_gp' in lite_pta.signals:
        labels = [r'$\Delta t_{\rm{Chr,1400}}$ ($\mu$s)'] + labels
        sig_names = [f'{psrname}_chrom_gp'] + sig_names
    lite_toas_only = np.isin(full_psr.toas, lite_psr.toas)
    for i in range(n_realizations):
        for sig_name in sig_names:
            lite_sig[sig_name][i] -= np.mean(lite_sig[sig_name][i])
            full_sig[sig_name][i] -= np.mean(full_sig[sig_name][i][lite_toas_only])
    fig, ax = plt.subplots(len(sig_names),1,figsize=(8,2*len(sig_names)),sharex=True)
    for i, sig_name in enumerate(sig_names):
        # Full
        med = np.median(full_sig[sig_name], axis=0)
        err = np.quantile(full_sig[sig_name], [0.16,0.84], axis=0)
        ax[i].plot(full_psr.toas/const.day, med, '.C0', ms=1)
        ax[i].fill_between(full_psr.toas/const.day, err[0], err[1], alpha=0.2, color='C0', label='DR2')
        # lite
        med = np.median(lite_sig[sig_name], axis=0)
        err = np.quantile(lite_sig[sig_name], [0.16,0.84], axis=0)
        ax[i].plot(lite_psr.toas/const.day, med, '.C1', ms=1)
        ax[i].fill_between(lite_psr.toas/const.day, err[0], err[1], alpha=0.2, color='C1',
                           label=f'DR2lite ({pta_label})')
        # labels
        ax[i].set_ylabel(labels[i])
        ax[i].grid(lw=0.3)
    ax[0].set_title(psrname)
    ax[0].legend()
    ax[1].set_xlabel(r'$t$ (MJD)')
    fig.tight_layout()
    fig.subplots_adjust(hspace=0)
    fig.savefig(f'{figsave_dir}/{psrname}_gps.png', dpi=300, bbox_inches='tight')
    plt.show()

J1909: Get signals and plot vs the TOAs

In [ ]:
n_realizations = 500
psrname = 'J1909-3744'
lite_psr = [p for p in lite_psrs if p.name == psrname][0]
full_psr = [p for p in full_psrs if p.name == psrname][0]
lite_pta = adv_noise_block(lite_psr, full_pta_analysis=False, dataset='lite', psr_model=False,
                           tm_marg=False, tm_svd=False)
full_pta = adv_noise_block(full_psr, full_pta_analysis=False, dataset='full', psr_model=False,
                           tm_marg=False, tm_svd=False)
pta_label = np.unique(lite_psr.flags['pta'])[0]
lite_pta.set_default_params(lite_noise_params)
full_pta.set_default_params(full_noise_params)
for signal in lite_pta.signals:
    lite_pta.signals[signal].basis_combine = False
for signal in full_pta.signals:
    full_pta.signals[signal].basis_combine = False
lite_sig = ppu.make_gp_realizations(lite_cores[psrname], lite_pta, lite_psr, all_dm=True,
                                    tm_params=['Offset','F0', 'F1', 'DM', 'DM1', 'DM2', 'NE_SW'],
                                    pardir=lite_pardir, n_realizations=n_realizations,
                                    use_dm_units=True, ref_MHz=1400)
lite_sig[f'{psrname}_red_noise'] += (lite_sig[f'{psrname}_Offset'] + lite_sig[f'{psrname}_F0'] +
                                     lite_sig[f'{psrname}_F1'])
if f'{psrname}_hf_red_noise' in lite_sig:
    lite_sig[f'{psrname}_red_noise'] += lite_sig[f'{psrname}_hf_red_noise']
full_sig = ppu.make_gp_realizations(full_cores[psrname], full_pta, full_psr, all_dm=True,
                                    tm_params=['Offset', 'F0', 'F1', 'DM', 'DM1', 'DM2', 'NE_SW'],
                                    pardir=full_pardir, n_realizations=n_realizations,
                                    use_dm_units=True, ref_MHz=1400)
full_sig[f'{psrname}_red_noise'] += (full_sig[f'{psrname}_Offset'] + full_sig[f'{psrname}_F0'] +
                                     full_sig[f'{psrname}_F1'])
if f'{psrname}_hf_red_noise' in full_sig:
    full_sig[f'{psrname}_red_noise'] += full_sig[f'{psrname}_hf_red_noise']

In [ ]:
lite_sig.keys()

In [ ]:
gp = ppu.ConditionalGP(lite_pta, tm_params=['DM','DM1','DM2'], psr=lite_psr)
params = lite_cores[psrname].map_params
lite_GPs = gp.sample_processes(params, n=1)[0]
# get red noise uncertainties
lite_red_noise = np.array([GP['J1909-3744_red_noise'] for GP in gp.sample_processes(params, n=1000)])
lite_GPs.keys(), lite_red_noise.shape

In [ ]:
correction = lite_GPs['J1909-3744_linear_timing_model']
pu.plot_resids(lite_psr, correction=correction, correction_label=' timing model (Lite)')

In [ ]:
correction = lite_GPs['J1909-3744_linear_timing_model'] + lite_GPs['J1909-3744_dm_gp']
fig = pu.plot_resids(lite_psr, correction=correction, correction_label=' timing model & DM (Lite)',
                     return_fig=True)
ax = fig.axes[0]
med = np.median(lite_red_noise, axis=0)
quantile = np.quantile(lite_red_noise, [0.16,0.84], axis=0)
ax.plot(lite_psr.toas/const.day, med*1e6, 'k')
ax.fill_between(lite_psr.toas/const.day, quantile[0]*1e6, quantile[1]*1e6, alpha=0.3, color='k')
fig.show()

In [ ]:
gp = ppu.ConditionalGP(full_pta, tm_params=['DM','DM1','DM2'], psr=full_psr)
params = full_cores[psrname].get_map_dict()
full_GPs = gp.sample_processes(params, n=1)[0]
# get red noise uncertainties
full_red_noise = np.array([GP['J1909-3744_red_noise'] for GP in gp.sample_processes(params, n=1000)])
full_GPs.keys(), full_red_noise.shape

In [ ]:
correction = full_GPs['J1909-3744_linear_timing_model']
PPTA_mask = full_psr.flags['pta'] == 'PPTA'
pu.plot_resids(full_psr, correction=correction, correction_label=' timing model (Full - PPTA only)',
               mask=PPTA_mask)

In [ ]:
correction = full_GPs['J1909-3744_linear_timing_model'] + full_GPs['J1909-3744_dm_gp']
fig = pu.plot_resids(full_psr, correction=correction, correction_label=' timing model & DM (Full - PPTA only)',
                     return_fig=True, mask=PPTA_mask)
ax = fig.axes[0]
med = np.median(full_red_noise, axis=0)
quantile = np.quantile(full_red_noise, [0.16,0.84], axis=0)
ax.plot(full_psr.toas/const.day, med*1e6, 'k')
ax.fill_between(full_psr.toas/const.day, quantile[0]*1e6, quantile[1]*1e6, alpha=0.3, color='k')
fig.show()

In [ ]:
# Here's the kicker - do the PPTA resids from Lite look like the Full resids
Lite_mask = np.isin(full_psr.toas, lite_psr.toas)
# if I subtract Full DR2's DM from Lite instead of Lite's DM?
correction = lite_GPs['J1909-3744_linear_timing_model'] + full_GPs['J1909-3744_dm_gp'][Lite_mask]
#correction -= lite_GPs['J1909-3744_DM1'] + lite_GPs['J1909-3744_DM2']
#correction += full_GPs['J1909-3744_DM1'][Lite_mask] + full_GPs['J1909-3744_DM2'][Lite_mask]
fig = pu.plot_resids(lite_psr, correction=correction, correction_label=' w/ timing model & FULL DM (Lite)',
                     return_fig=True)
ax = fig.axes[0]
ax.plot(full_psr.toas/const.day, med*1e6, 'k')
ax.fill_between(full_psr.toas/const.day, quantile[0]*1e6, quantile[1]*1e6, alpha=0.3, color='k')
fig.show()

In [ ]:
tm_dt = lite_sig['J1909-3744_linear_timing_model']
dm_dt = lite_sig['J1909-3744_dm_gp']/(1e15*const.DM_K*lite_psr.freqs**2)*1e6
all_dt = lite_sig['J1909-3744_all']*1e6
tm_dm_dt_med = np.median(tm_dt + dm_dt, axis=0)
rn_dt = lite_sig['J1909-3744_red_noise']
all_dt_med = np.median(all_dt, axis=0)
all_mrn_dt_med = np.median(all_dt - rn_dt, axis=0)
lite_correction = all_mrn_dt_med/1e6

In [ ]:
# these are the Lite resids without RN corrections
pu.plot_resids(lite_psr, correction=lite_correction, correction_label=' Lite')

In [ ]:
all_dt = full_sig['J1909-3744_all']*1e6
rn_dt = full_sig['J1909-3744_red_noise']
all_mrn_dt_med = np.median(all_dt - rn_dt, axis=0)
rn_dt -= np.mean(rn_dt)
full_correction = all_mrn_dt_med/1e6
full_red_noise = np.median(rn_dt, axis=0)
full_red_noise_err = np.quantile(rn_dt, [0.16,0.84], axis=0)
full_red_noise_err.shape

In [ ]:
# these are the Full resids without RN corrections (PPTA only) + RN overplotted
mask = full_psr.flags['pta'] == 'PPTA'
fig = pu.plot_resids(full_psr, correction=full_correction, mask=mask, correction_label=' Full', return_fig=True)
ax = fig.axes[0]
ax.plot(full_psr.toas/const.day, full_red_noise, '-k')
ax.fill_between(full_psr.toas/const.day, full_red_noise_err[0], full_red_noise_err[1], alpha=0.2, color='k')

In [ ]:
# these are the Full resids without RN corrections (EPTA only) + RN overplotted
mask = full_psr.flags['pta'] == 'EPTA'
fig = pu.plot_resids(full_psr, correction=full_correction, mask=mask, correction_label=' Full', return_fig=True)
ax = fig.axes[0]
ax.plot(full_psr.toas/const.day, full_red_noise, '-k')
ax.fill_between(full_psr.toas/const.day, full_red_noise_err[0], full_red_noise_err[1], alpha=0.2, color='k')

In [ ]:
# these are the Full resids without RN corrections (NG only) + RN overplotted
mask = full_psr.flags['pta'] == 'NANOGrav'
fig = pu.plot_resids(full_psr, correction=full_correction, mask=mask, correction_label=' Full', return_fig=True)
ax = fig.axes[0]
ax.plot(full_psr.toas/const.day, full_red_noise, '-k')
ax.fill_between(full_psr.toas/const.day, full_red_noise_err[0], full_red_noise_err[1], alpha=0.2, color='k')

In [ ]:
# these are the Lite resids if you assume the DM variations from Full instead of Lite
mask = full_psr.flags['pta'] == 'PPTA'
lite_dm_dt = lite_sig['J1909-3744_dm_gp']*1e6/(1e15*const.DM_K*lite_psr.freqs**2)
full_dm_dt = full_sig['J1909-3744_dm_gp']*1e6/(1e15*const.DM_K*full_psr.freqs**2)
#pu.plot_resids(lite_psr, correction=lite_correction, correction_label=' Lite')

In [ ]:
# plot
labels = [r'$\Delta$DM ($10^{-3}$ pc/cm$^3$)', r'$\Delta t_{\rm{RN}}$ ($\mu$s)']
sig_names = [f'{psrname}_dm_gp', f'{psrname}_red_noise']
if f'{psrname}_chrom_gp' in lite_pta.signals:
    labels = [r'$\Delta t_{\rm{Chr,1400}}$ ($\mu$s)'] + labels
    sig_names = [f'{psrname}_chrom_gp'] + sig_names
lite_toas_only = np.isin(full_psr.toas, lite_psr.toas)
for i in range(n_realizations):
    for sig_name in sig_names:
        lite_sig[sig_name][i] -= np.mean(lite_sig[sig_name][i])
        full_sig[sig_name][i] -= np.mean(full_sig[sig_name][i][lite_toas_only])
sig_names += [f'{psrname}_all']
labels += [r'$\Delta t_{\rm{all}}$ ($\mu$s)']
fig, ax = plt.subplots(len(sig_names),1,figsize=(8,2*len(sig_names)),sharex=True)
for i, sig_name in enumerate(sig_names):
    # Full
    med = np.median(full_sig[sig_name], axis=0)
    err = np.quantile(full_sig[sig_name], [0.16,0.84], axis=0)
    ax[i].plot(full_psr.toas/const.day, med, '.C0', ms=1)
    ax[i].fill_between(full_psr.toas/const.day, err[0], err[1], alpha=0.2, color='C0', label='DR2')
    # lite
    med = np.median(lite_sig[sig_name], axis=0)
    err = np.quantile(lite_sig[sig_name], [0.16,0.84], axis=0)
    ax[i].plot(lite_psr.toas/const.day, med, '.C1', ms=1)
    ax[i].fill_between(lite_psr.toas/const.day, err[0], err[1], alpha=0.2, color='C1',
                       label=f'DR2lite ({pta_label})')
    # labels
    ax[i].set_ylabel(labels[i])
    ax[i].grid(lw=0.3)
ax[0].set_title(psrname)
ax[0].legend()
ax[1].set_xlabel(r'$t$ (MJD)')
fig.tight_layout()
fig.subplots_adjust(hspace=0)
#fig.savefig(f'{figsave_dir}/{psrname}_gps_withtotal.png', dpi=300, bbox_inches='tight')
plt.show()

## Compare GP spectra

In [ ]:
def get_freqs(pta, signal_id='red_noise'):
    """ Hackish way to get frequency vector."""
    for sig in pta._signalcollections[0]._signals:
        if sig.signal_id == signal_id:
            sig._construct_basis()
            try:
                freqs = np.array(sig._labels[''])[::2]
            except IndexError:
                freqs = sig._labels[::2]
            break
    return freqs

In [ ]:
n_realizations = 1000
for psrname in lite_psrnames[-3:]:
    # setup ptas
    lite_psr = [p for p in lite_psrs if p.name == psrname][0]
    full_psr = [p for p in full_psrs if p.name == psrname][0]
    pta_label = np.unique(lite_psr.flags['pta'])[0]
    lite_pta = adv_noise_block(lite_psr, full_pta_analysis=False, dataset='lite', psr_model=False,
                               tm_marg=False, tm_svd=False)
    full_pta = adv_noise_block(full_psr, full_pta_analysis=False, dataset='full', psr_model=False,
                               tm_marg=False, tm_svd=False)
    pta_label = np.unique(lite_psr.flags['pta'])[0]
    lite_pta.set_default_params(lite_noise_params)
    full_pta.set_default_params(full_noise_params)
    for signal in lite_pta.signals:
        lite_pta.signals[signal].basis_combine = False
    for signal in full_pta.signals:
        full_pta.signals[signal].basis_combine = False
    # setup labels
    labels = [r'$\Delta$DM ($10^{-3}$ pc/cm$^3$)', r'$\Delta t_{\rm{RN}}$ ($\mu$s)']
    sig_names = [f'{psrname}_dm_gp', f'{psrname}_red_noise']
    if f'{psrname}_chrom_gp' in lite_pta.signals:
        labels = [r'$\Delta t_{\rm{Chr,1400}}$ ($\mu$s)'] + labels
        sig_names = [f'{psrname}_chrom_gp'] + sig_names
    # get Fourier coefficients - lite
    gp = ConditionalGP(lite_pta)
    lite_rho = {}
    for i in tqdm.tqdm(range(n_realizations)):
        params = {p:np.random.choice(lite_cores[psrname](p)) for p in lite_cores[psrname].params}
        try:
            GPs = gp.sample_coefficients(params, n=1)[0]
        except LinAlgError:
            print('Ran into cholesky error')
            lite_rho[name][i] = lite_rho[name][i-1]
            continue
        for name in sig_names:
            Nf = len(get_freqs(lite_pta, signal_id='_'.join(name.split('_')[1:])))
            if i == 0:
                lite_rho[name] = np.zeros((n_realizations,Nf))
            a = GPs[f'{name}_coefficients']
            for j in range(Nf):
                lite_rho[name][i,j] = np.sqrt(a[j]**2 + a[j+1]**2)
    # get Fourier coefficients - adv noise
    gp = ConditionalGP(full_pta)
    full_rho = {}
    for i in tqdm.tqdm(range(n_realizations)):
        params = {p:np.random.choice(full_cores[psrname](p)) for p in full_cores[psrname].params}
        try:
            GPs = gp.sample_coefficients(params, n=1)[0]
        except LinAlgError:
            print('Ran into cholesky error')
            full_rho[name][i] = full_rho[name][i-1]
            continue
        for name in sig_names:
            Nf = len(get_freqs(full_pta, signal_id='_'.join(name.split('_')[1:])))
            if i == 0:
                full_rho[name] = np.zeros((n_realizations,Nf))
            a = GPs[f'{name}_coefficients']
            for j in range(Nf):
                full_rho[name][i,j] = np.sqrt(a[j]**2 + a[j+1]**2)
    # plot
    fig, ax = plt.subplots(len(sig_names),1,figsize=(8,2*len(sig_names)),sharex=True)
    for i, name in enumerate(sig_names):
        # DR2 full
        freqs = get_freqs(full_pta, signal_id='_'.join(name.split('_')[1:]))
        med = np.median(full_rho[name], axis=0)
        err = np.quantile(full_rho[name], [0.05,0.95], axis=0)
        ax[i].plot(freqs, med, '-C0', ms=1, label='DR2')
        ax[i].fill_between(freqs, err[0], err[1], alpha=0.2, color='C0')
        # DR2 lite
        freqs = get_freqs(lite_pta, signal_id='_'.join(name.split('_')[1:]))
        med = np.median(lite_rho[name], axis=0)
        err = np.quantile(lite_rho[name], [0.05,0.95], axis=0)
        ax[i].plot(freqs, med, '-C1', ms=1, label=f'DR2 lite ({pta_label})')
        ax[i].fill_between(freqs, err[0], err[1], alpha=0.2, color='C1')
        # labels
        ax[i].set_ylabel(labels[i])
        ax[i].grid(which='both', lw=0.3)
        ax[i].loglog()
        ax[i].axvline(1/const.yr, alpha=0.5, ls='--', color='k')
        ax[i].set_ylim([1.01e-10, 1e-4])
        if psrname == 'J1939+2134':
            ax[i].set_ylim([1.01e-10, 1e-3])
    ax[0].set_title(psrname)
    ax[0].legend()
    ax[1].set_xlabel(r'$f$ (Hz)')
    fig.tight_layout()
    fig.subplots_adjust(hspace=0)
    fig.savefig(f'{figsave_dir}/{psrname}_spectra.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
psr_paths

In [ ]:
import shutil

In [ ]:
for path in psr_paths:
    src = path.replace('lite_ePSRs','full_ePSRs')
    dst = path.replace('lite_ePSRs','litecombined_ePSRs')
    shutil.copyfile(src, dst)

In [ ]:
! ls /vast/palmer/home.grace/bbl29/IPTA_DR2_analysis/data/

In [ ]:
figsave_dir